# FIT5196 Assessment 2 Task 2
#### Student Name: Dariush Riazati
#### Student ID: 11230991

Date: 09/06/2019

Version: 1.0

Environment: Python 3.5.2 and Jupyter notebook

In [1]:
from sys import version
print(version)

3.5.2 (v3.5.2:4def2a2901a5, Jun 25 2016, 22:18:55) [MSC v.1900 64 bit (AMD64)]


In [2]:
import pandas as pd
import re
import csv

# Functions

In [3]:
company_match_file = True

In [4]:
def cleanToken(token):
    if pd.isnull(token):
        return token
    else:
        # normalize to upper case letters
        token = token.upper()       
        # remove all special characters except space. +, / and dot
        token = re.sub(r'([^\w\s\.\/\+]|_)', '', token)
        token = re.sub('^\s+', '', token)
        # replace multiple spaces with a single space, also trim spaces on both side
        token = re.sub( '\s+', ' ', token).strip()
        return token

In [5]:
def reformat_date(dt):
    y = re.findall('[\d]{4}', dt)[0]
    m = re.findall('(?:[\d]{4}-)([\d]{2})', dt)[0]
    d = re.findall('(?:[\d]{4}-)(?:[\d]{2}-)([\d]{2})', dt)[0]
    h = re.findall('(?:[\d]{4}-)(?:[\d]{2}-)(?:[\d]{2} )([\d]{2})', dt)[0]
    m = re.findall('(?:[\d]{4}-)(?:[\d]{2}-)(?:[\d]{2} )(?:[\d]{2}:)([\d]{2})', dt)[0]
    s = re.findall('(?:[\d]{4}-)(?:[\d]{2}-)(?:[\d]{2} )(?:[\d]{2}:)(?:[\d]{2}:)([\d]{2})', dt)[0]
    return y+m+d+'T'+h+m+s

# Main

In [6]:
# Note that based on the values, the COntractType in data set 2 is much closer to COntractTime in data set 2
# and hence we are re-labeling ContractType as ContractTime 
# We are also re-labeling Fraction as ContractType

In [7]:
df1 = pd.read_csv('dataset1_solution.csv', sep=",", names=['Id', 'Title', 'Location', 'Company', 'ContractType', 
                                                            'ContractTime', 'Category', 'Salary', 'OpenDate', 'CloseDate', 'SourceName'],
                            encoding = "utf-8", skiprows=1)

df2 = pd.read_csv('dataset2.csv', sep=",", names=['OpenDate', 'CloseDate', 'Title', 'Company',
                        'Location', 'Category', 'Salary', 'ContractType', 'ContractTime'],
                            encoding = "utf-8", skiprows=1)

### Align Schemas

In [8]:
df1_cols = df1.columns.difference([])
df2_cols = df2.columns.difference([])

set(df1_cols) - set(df2_cols)

{'Id', 'SourceName'}

In [9]:
del df1['Id']
del df1['SourceName']

In [10]:
print("DF1:", df1.shape)
print("DF2:", df2.shape)

DF1: (55166, 9)
DF2: (334, 9)


In [11]:
df1['Salary'] = df1['Salary'].astype(float)
df2['Salary'] = df2['Salary'].astype(float)

In [12]:
print(df1.info())
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55166 entries, 0 to 55165
Data columns (total 9 columns):
Title           55166 non-null object
Location        55166 non-null object
Company         55166 non-null object
ContractType    55166 non-null object
ContractTime    55166 non-null object
Category        55166 non-null object
Salary          55166 non-null float64
OpenDate        55166 non-null object
CloseDate       55166 non-null object
dtypes: float64(1), object(8)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 9 columns):
OpenDate        334 non-null object
CloseDate       334 non-null object
Title           334 non-null object
Company         334 non-null object
Location        334 non-null object
Category        334 non-null object
Salary          334 non-null float64
ContractType    3 non-null object
ContractTime    327 non-null object
dtypes: float64(1), object(8)
memory usage: 23.6+ KB
None


In [13]:
df1_cols = df1.columns.difference([])
df2_cols = df2.columns.difference([])

set(df1_cols) - set(df2_cols)

set()

In [14]:
for c in df1_cols:
    if df1[c].dtype == df2[c].dtype:
       print("Matching data type for", c)
    else:
       print("Mismatching data type for", c)

Matching data type for Category
Matching data type for CloseDate
Matching data type for Company
Matching data type for ContractTime
Matching data type for ContractType
Matching data type for Location
Matching data type for OpenDate
Matching data type for Salary
Matching data type for Title


In [15]:
df1.Salary.dtype

dtype('float64')

## Reconcile OpenDate

In [16]:
df1.OpenDate.head(5)
df2.OpenDate.head(5)

0    2013-10-06 00:00:00
1    2012-10-03 12:00:00
2    2012-01-01 00:00:00
3    2012-10-14 00:00:00
4    2012-11-17 12:00:00
Name: OpenDate, dtype: object

In [17]:
df2['OpenDate'] = df2.OpenDate.apply(lambda x: reformat_date(x))

In [18]:
df2['OpenDate'].head(3)

0    20130006T000000
1    20120003T120000
2    20120001T000000
Name: OpenDate, dtype: object

### Reconcile CloseDate

In [19]:
df1.CloseDate.head(5)
df2.CloseDate.head(5)

0    2013-12-05 00:00:00
1    2012-11-02 12:00:00
2    2012-01-31 00:00:00
3    2012-11-13 00:00:00
4    2013-01-16 12:00:00
Name: CloseDate, dtype: object

In [20]:
df2['CloseDate'] = df2.CloseDate.apply(lambda x: reformat_date(x))

In [21]:
df2['CloseDate'].head(3)

0    20130005T000000
1    20120002T120000
2    20120031T000000
Name: CloseDate, dtype: object

### Reconcile Title

In [22]:
df1.Title.head(5)
df2.Title.head(5)

0                        Aviation loans administration
1               Payroll Analyst  City upto **** , ****
2    Investment Team Assistant for leading Private ...
3                     SWAPS COLLATERAL CONTROL OFFICER
4                           Loans Administration  Temp
Name: Title, dtype: object

In [23]:
df2['Title'] = df2.Title.apply(lambda x: cleanToken(x))

In [24]:
df1.Title.head(5)
df2.Title.head(5)

0                        AVIATION LOANS ADMINISTRATION
1                            PAYROLL ANALYST CITY UPTO
2    INVESTMENT TEAM ASSISTANT FOR LEADING PRIVATE ...
3                     SWAPS COLLATERAL CONTROL OFFICER
4                            LOANS ADMINISTRATION TEMP
Name: Title, dtype: object

### Reconcile Company

In [25]:
df1.Company.head(5)
df2.Company.head(5)
df2.Company.isnull().sum()

0

In [26]:
df2['Company'] = df2.Company.apply(lambda x: cleanToken(x))

In [27]:
df1.Company.head(5)
df2.Company.head(5)

0                                CER FINANCIAL
1                          LMA RECRUITMENT LTD
2                            AUSTIN ANDREW LTD
3    BRIAN DURHAM RECRUITMENT SERVICES LIMITED
4                                CER FINANCIAL
Name: Company, dtype: object

In [28]:
df2['Company'] = df2.Company.str.replace('LTD', 'LIMITED')

In [29]:
df1.Company.head(5)
df2.Company.head(5)

0                                CER FINANCIAL
1                      LMA RECRUITMENT LIMITED
2                        AUSTIN ANDREW LIMITED
3    BRIAN DURHAM RECRUITMENT SERVICES LIMITED
4                                CER FINANCIAL
Name: Company, dtype: object

### Reconcile ContractType

In [30]:
df1.ContractType.value_counts()
df2.ContractType.value_counts()
len(df2[df2.ContractType.isnull()])


331

In [31]:
df2.ContractType.value_counts()

Part Time    3
Name: ContractType, dtype: int64

In [32]:
df2['ContractType'] = df2.ContractType.fillna('non-specified')
len(df2[df2.ContractType.isnull()])

0

In [33]:
df1.ContractType.value_counts()
df2.ContractType.value_counts()

non-specified    331
Part Time          3
Name: ContractType, dtype: int64

In [34]:
df2['ContractType'] = df2.ContractType.str.replace('Part Time', 'part_time')
df1.ContractType.value_counts()
df2.ContractType.value_counts()

non-specified    331
part_time          3
Name: ContractType, dtype: int64

### Reconcile ContractTime

In [35]:
#Based on the values of Contract types between the two sources, it appears that the contact type 
# in data set 2 is in fact contract time.

In [36]:
df1.ContractTime.value_counts()
df2.ContractTime.value_counts()
len(df2[df2.ContractTime.isnull()])

7

In [37]:
# There is not sufficient evidence to suggest that the null values are in fact full_time.

In [38]:
df2['ContractTime'] = df2.ContractTime.fillna('non-specified')
df1.ContractTime.value_counts()
df2.ContractTime.value_counts()
len(df2[df2.ContractTime.isnull()])

0

### Reconcile Location

In [39]:
df1.Location.value_counts()
df2.Location.value_counts()
len(df2[df2.Location.isnull()])

0

In [40]:
df2['Location'] = df2.Location.apply(lambda x: cleanToken(x))
df2.Location.value_counts()

LONDON                158
THE CITY               92
UK                     25
CITY                   13
SOUTH EAST ENGLAND      5
WEST END                5
CENTRAL LONDON          3
SOUTH WEST LONDON       3
HERTFORDSHIRE           2
BIRMINGHAM              2
CROYDON                 2
CRAWLEY                 2
NORTH LONDON            1
NORTH WEST ENGLAND      1
PETERBOROUGH            1
BRISTOL                 1
WEST SUSSEX             1
READING                 1
DORKING                 1
HAMPSHIRE               1
WEST LONDON             1
EAST MIDLANDS           1
BATH                    1
SOUTHAMPTON             1
FAREHAM                 1
BERKSHIRE               1
GUILDFORD               1
CHELMSFORD              1
BASINGSTOKE             1
SOUTH EAST LONDON       1
REDHILL                 1
BUCKINGHAMSHIRE         1
CHELTENHAM              1
FARNHAM                 1
Name: Location, dtype: int64

### Reconcile Category

In [41]:
df1.Category.value_counts()
df2.Category.value_counts()
len(df2[df2.Category.isnull()])

0

In [42]:
#df2['Category'] = df2.Category.apply(lambda x: cleanToken(x))
df2['Category'] = df2.Category.str.replace(' and ', ' & ')
df1.Category.value_counts()
df2.Category.value_counts()

Finance & Accounting           319
Information Technology          12
PR, Advertising & Marketing      3
Name: Category, dtype: int64

In [43]:
df1['Category'] = df1.Category.str.replace(' Jobs', '')
df2['Category'] = df2.Category.str.replace('Finance & Accounting', 'Accounting & Finance')
df2['Category'] = df2.Category.str.replace('Information Technology', 'IT')
df1.Category.value_counts()
df2.Category.value_counts()

Accounting & Finance           319
IT                              12
PR, Advertising & Marketing      3
Name: Category, dtype: int64

### Reconcile Salary

In [44]:
df1.Salary.head(5)
df2.Salary.head(5)

0    2800.0
1    2917.0
2    3750.0
3    3333.0
4    3280.0
Name: Salary, dtype: float64

In [45]:
# Based on the column name, the salary is based on a monthly basis. We multiply by 12 to get annual salary consistent with 
# the data set 1

In [46]:
df2['Salary'] = df2.Salary.apply(lambda x: x*12)

In [47]:
df2.Salary.head(5)

0    33600.0
1    35004.0
2    45000.0
3    39996.0
4    39360.0
Name: Salary, dtype: float64

### Combine data sets & remove duplicates

In [48]:
df = pd.concat([df1,df2], sort=True)
df['Salary'] = df.Salary.astype(float)

In [49]:
df.columns.difference([])

Index(['Category', 'CloseDate', 'Company', 'ContractTime', 'ContractType',
       'Location', 'OpenDate', 'Salary', 'Title'],
      dtype='object')

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55500 entries, 0 to 333
Data columns (total 9 columns):
Category        55500 non-null object
CloseDate       55500 non-null object
Company         55500 non-null object
ContractTime    55500 non-null object
ContractType    55500 non-null object
Location        55500 non-null object
OpenDate        55500 non-null object
Salary          55500 non-null float64
Title           55500 non-null object
dtypes: float64(1), object(8)
memory usage: 4.2+ MB


In [51]:
cols = df.columns.difference([])
duplicates = df[df.duplicated(cols,keep=False)]
print( "There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=duplicates.columns.tolist()).head(10)

There are 0 duplicate records found


,Category,CloseDate,Company,ContractTime,ContractType,Location,OpenDate,Salary,Title


In [52]:
#df = df.drop_duplicates(cols, keep='last')
df.shape

(55500, 9)

In [53]:
# Making sure duplicates are droped.
duplicates = df[df.duplicated(cols,keep=False)]
print( "There are "+ str(len(duplicates)) + " duplicate records found")

There are 0 duplicate records found


In [54]:
# check duplication with respect to the key columns being title, open date, close date and company.
# However, we leave out company to see what we get.
cols = ['Title','OpenDate', 'CloseDate'] 
duplicates = df[df.duplicated(cols,keep=False)]
print( "There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=duplicates.columns.tolist()).head(10)

There are 2 duplicate records found


,Category,CloseDate,Company,ContractTime,ContractType,Location,OpenDate,Salary,Title
34674,Accounting & Finance,20130225T000000,FRAM EXECUTIVE SEARCH,permanent,non-specified,LONDON,20130126T000000,40000.0,SENIOR FINANCIAL ADVISOR
55163,Accounting & Finance,20130225T000000,FRAM EXECUTIVE SEARCH.,permanent,non-specified,LONDON,20130126T000000,40000.0,SENIOR FINANCIAL ADVISOR


In [55]:
# The company in the second row has an dot at the end.
df = df.drop_duplicates(cols, keep='first')

In [56]:
# Making sure duplicates are droped.
cols = ['Title','OpenDate', 'CloseDate'] 
duplicates = df[df.duplicated(cols,keep=False)]
print( "There are "+ str(len(duplicates)) + " duplicate records found")

There are 0 duplicate records found


###### Based on the above, we do not need to include other columns in the duplicate validation as tehy are bound to return no duplicate.

In [57]:
# Resequence columns
df = df[['Title', 'Location', 'Company', 'ContractType', 'ContractTime', 'Category', 'Salary', 'OpenDate', 'CloseDate']]

In [58]:
df.to_csv('dataset_integrated.csv', index =False)